In [2]:
import pymysql

connection = pymysql.connect(
    host='localhost',
    user='kita',
    password='kita',
    db='kita_db',
)

cursor = connection.cursor()

RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods

In [12]:
import pymysql

connection = pymysql.connect(
    host='localhost',
    user='kita',
    password='kita',
    db='kita_db'
)

cursor = connection.cursor()

## DDL

In [3]:
#테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS customer (
    id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) NOT NULL,
    age INT
)
"""
cursor.execute(create_table_query)
connection.commit()

In [13]:
create_table_query = """
CREATE TABLE IF NOT EXISTS customer (
    id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) NOT NULL,
    age INT
)
"""
cursor.execute(create_table_query)
connection.commit()


In [14]:
#컬럼 추가
#이 컬럼은 timestamp 형식이며, 기본값은 현재 시간이다.기본값으로 현재 타임스템프가 설정, 즉 , 데이터가 추가될 때마다 현재 시간이 기록된다.
alter_table_query = "ALTER TABLE customer ADD COLUMN created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP"
cursor.execute(alter_table_query)
connection.commit()

In [15]:
#email 컬럼에 not null 제약 조건 제거
alter_table_query = "ALTER TABLE customer MODIFY email VARCHAR(100)"
cursor.execute(alter_table_query)
connection.commit()

In [10]:
#테이블 삭제
drop_table_query = "DROP TABLE if exists customer"
cursor.execute(drop_table_query)
connection.commit()

In [11]:
#연결 종료
cursor.close()
connection.close()

In [ ]:
#대량 데이터 삽입을 위해 위와 같은 과정 필요

## DML

In [20]:
#데이터 입력
# %s는 mysql.connector에서 사용하는 플레이스홀더이다. 자리 표시자, 이 플레이스홀더는 데이터베이스에 데이터를 입력할 때 사용된다.(sqlite3에서는 ?를 사용)
insert_query = "INSERT INTO customer (username, email, age) VALUES (%s, %s, %s)"
data = ("john_doe2", "john@example.com", 30)
cursor.execute(insert_query, data)
connection.commit()

In [17]:
#데이터 수정
select_query = "SELECT * FROM customer"
cursor.execute(select_query)
result = cursor.fetchall()
for row in result:
    print(row)

(1, 'john_doe', 'john@example.com', 30, datetime.datetime(2024, 7, 15, 12, 11, 25))


In [18]:
#데이터 수정
update_query = "UPDATE customer SET age = %s WHERE username = %s"
cursor.execute(update_query, (31, "john_doe"))
connection.commit()

In [23]:
select_query = "SELECT * FROM customer"
cursor.execute(select_query)
result = cursor.fetchall()
for row in result:
    print(row)

(2, 'john_doe2', 'john@example.com', 30, datetime.datetime(2024, 7, 15, 12, 14, 41))


In [22]:
#data delete
delete_query = "DELETE FROM customer WHERE username = %s"
cursor.execute(delete_query, ("john_doe"))
connection.commit()

In [24]:
#데이터 반복 삽입, 각 항목은 리스트나 튜플과 같은 순서가 있는 데이터 구조이어야 합니다.

customers = [
    ("alice", "alice@example.com", 25),
    ("bob", "bob@example.com", 28),
    ("charlie", "chalie@example.com",22)
]

cursor.executemany(insert_query, customers)
connection.commit()

In [2]:
select_query = "SELECT * from customer"
cursor.execute(select_query)
result = cursor.fetchall()
for row in result:
    print(row)

(2, 'john_doe2', 'john@example.com', 30, datetime.datetime(2024, 7, 15, 12, 14, 41))
(3, 'alice', 'alice@example.com', 25, datetime.datetime(2024, 7, 15, 12, 20, 21))
(4, 'bob', 'bob@example.com', 28, datetime.datetime(2024, 7, 15, 12, 20, 21))
(5, 'charlie', 'chalie@example.com', 22, datetime.datetime(2024, 7, 15, 12, 20, 21))


In [27]:
#데이터 조회 결과를 dataframe으로 변환
import pandas as pd

column_names = [i[0] for i in cursor.description]
# print(cursor.description)
df = pd.DataFrame(result, columns=column_names)
print(df)

   id   username               email  age          created_at
0   2  john_doe2    john@example.com   30 2024-07-15 12:14:41
1   3      alice   alice@example.com   25 2024-07-15 12:20:21
2   4        bob     bob@example.com   28 2024-07-15 12:20:21
3   5    charlie  chalie@example.com   22 2024-07-15 12:20:21


In [28]:
#DataFrame을 csv 파일로 저장
df.to_csv("customers.csv", index=False, encoding='utf-8-sig')

In [29]:
#buyer 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS buyer (
    id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) NOT NULL,
    age INT
)
"""
cursor.execute(create_table_query)
connection.commit()

In [30]:
#csv 파일에서 데이터 읽어오기
import csv

file_path = "customers.csv"
df = pd.read_csv(file_path)
#필요한 컬럼만 선택
data_to_insert = df[['username', 'email', 'age']].values.tolist()
# 쿼리 정의
insert_query = "INSERT INTO buyer (username, email, age) VALUES (%s, %s, %s)"
# 데이터 삽입
cursor.executemany(insert_query, data_to_insert)
connection.commit()

In [32]:
#데이터 조회 및 dataframe으로 변환
select_query = "SELECT * FROM buyer"
cursor.execute(select_query)
result = cursor.fetchall()
#컬럼 이름 가져오기
column_names = [i[0] for i in cursor.description]
#데이터프레임으로 변환
df = pd.DataFrame(result, columns=column_names)
print(df)
#연결 종료
cursor.close()
connection.close()

   id   username               email  age
0   1  john_doe2    john@example.com   30
1   2      alice   alice@example.com   25
2   3        bob     bob@example.com   28
3   4    charlie  chalie@example.com   22


In [3]:
print(cursor.description)

(('id', 3, None, 11, 11, 0, False), ('username', 253, None, 200, 200, 0, False), ('email', 253, None, 400, 400, 0, True), ('age', 3, None, 11, 11, 0, True), ('created_at', 7, None, 19, 19, 0, True))
